In [6]:
import sys

# append root project directory so packages will be available
sys.path.append('../..')

In [7]:
import src.utility.symbols_loader as sl
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.nn import functional as F
import einops
import os
import numpy as np
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision import transforms
from tqdm import tqdm
from pathlib import Path
import datetime
from src.architectures.course_autoencoder import Autoencoder, Encoder, Decoder
from src.architectures.letters_dataset import LettersDataset


In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True # should improve speed if input size don't change

In [9]:
LATENT_WIDTH = 16
FILE_PATH = Path().parent.as_posix() + "/"
path = FILE_PATH + "../../data/models/"

In [ ]:
trials = ['trial_0', 'trial_1', 'trial_2', 'trial_3', 'trial_4']

for trial in trials:
    emnist_Autoencoder = Autoencoder(Encoder(LATENT_WIDTH), Decoder(LATENT_WIDTH))
    emnist_Autoencoder.load_state_dict(torch.load(path + trial +'_emnist_autoencoder.pth'))
    emnist_Encoder = emnist_Autoencoder.encoder
    emnist_Encoder.to(device)
    mnist_dataset = LettersDataset(sl.load_emnist_pages(number_of_pages=5, trial=trial))

    emnist_val_loader = DataLoader(
        mnist_dataset,
        batch_size=32,
        num_workers=2,
        shuffle=True,
        pin_memory=True,
    )

    loss_func = F.l1_loss
    optimizer = torch.optim.Adam(emnist_Autoencoder.parameters(), lr=1e-3)

    emnist_preds = []

    with torch.no_grad() as nograd:
        for batch in tqdm(emnist_val_loader):

            images = batch.to(device)
            predictions = emnist_Encoder(images)

            emnist_preds.append(predictions.cpu())

    emnist_preds_numpy = np.zeros((len(emnist_preds) * 32, LATENT_WIDTH))

    iterator = 0
    for tensor in emnist_preds:
        for block in range(tensor.shape[0]):
            emnist_preds_numpy[iterator] = tensor[block].numpy().copy()
            iterator += 1

    path = FILE_PATH + "../../data/encoded_data/" + trial + '/'
    np.savez_compressed(path + "emnist_preds.npz", emnist_preds_numpy)

  0%|          | 0/405 [00:00<?, ?it/s]

In [ ]:
trials = ['trial_0', 'trial_1', 'trial_2', 'trial_3', 'trial_4']

for trial in trials:
    kuzushiji_Autoencoder = Autoencoder(Encoder(LATENT_WIDTH), Decoder(LATENT_WIDTH))
    kuzushiji_Autoencoder.load_state_dict(torch.load(path + trial +'/_kuzushiji_autoencoder.pth'))
    kuzushiji_Encoder = kuzushiji_Autoencoder.encoder
    kuzushiji_Encoder.to(device)
    kuzushiji_dataset = LettersDataset(sl.load_kuzushiji_pages(number_of_pages=5, trial=trial))

    kuzushiji_val_loader = DataLoader(
        kuzushiji_dataset,
        batch_size=32,
        num_workers=2,
        shuffle=True,
        pin_memory=True,
    )

    loss_func = F.l1_loss
    optimizer = torch.optim.Adam(kuzushiji_Autoencoder.parameters(), lr=1e-3)

    kuzushiji_preds = []

    with torch.no_grad() as nograd:
        for batch in tqdm(kuzushiji_val_loader):

            images = batch.to(device)
            predictions = kuzushiji_Encoder(images)

            kuzushiji_preds.append(predictions.cpu())

    kuzushiji_preds_numpy = np.zeros((len(kuzushiji_preds) * 32, LATENT_WIDTH))

    iterator = 0
    for tensor in kuzushiji_preds:
        for block in range(tensor.shape[0]):
            kuzushiji_preds_numpy[iterator] = tensor[block].numpy().copy()
            iterator += 1

    path = FILE_PATH + "../../data/encoded_data/" + trial + '/'
    np.savez_compressed(path + "kuzushiji_preds.npz", kuzushiji_preds_numpy)